In [ ]:
import optuna
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import halfnorm
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
import statsmodels.api as sm
import pandas as pd
import math
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import Lasso
import seaborn as sns

# Stochastic Frontier Model类
class StochasticFrontierModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.beta = nn.Parameter(torch.tensor(num_features * [1.0], dtype=torch.float32))
        self.log_sigma2 = nn.Parameter(torch.tensor(0.0))
        self.log_lambda0 = nn.Parameter(torch.tensor(0.0))

    def predict(self, x):
        # 使用 torch.matmul 执行矩阵乘法
        predictions = torch.matmul(x, self.beta)
        return predictions

    def forward(self, x, y):
        sigma2 = torch.exp(self.log_sigma2)
        lambda0 = torch.exp(self.log_lambda0)
        sigma = torch.sqrt(sigma2)
        epsilon = y - torch.sum(x * self.beta, dim=1)

        term1 = x.shape[0] * self.log_sigma2 / 2
        term2 = -torch.sum(torch.log(torch.distributions.normal.Normal(0, 1).cdf(-epsilon * lambda0 / sigma) + 1e-10))
        term3 = torch.sum(epsilon**2) / (2 * sigma2)

        return (term1 + term2 + term3) / x.shape[0]

class StochasticFrontierAnalysis:
    def __init__(self, num_features, sigma_u, sigma_v):
        self.num_features = num_features
        self.sigma_u = sigma_u
        self.sigma_v = sigma_v

    def standardize_data(self, x):
        mean = np.mean(x[:, 1:], axis=0)
        std = np.std(x[:, 1:], axis=0)
        x_standardized = np.column_stack([x[:, 0], (x[:, 1:] - mean) / std])
        return x_standardized, np.insert(mean, 0, 0), np.insert(std, 0, 1)

    def generate_data(self, N, save_path='true_parameters.csv'):
        """
        生成数据，并保存真实的 beta、lambda、sigma_sq 等参数到 CSV 文件。
        :param N: 样本数量
        :param save_path: 保存真实参数的 CSV 文件路径
        :return: x, y, true_beta
        """
        # 生成稀疏的 beta 向量（500 维，只有 6 个非零系数，取值为 1 或 -1）
        beta = np.zeros(self.num_features)
        non_zero_indices = np.random.choice(self.num_features, 6, replace=False)  # 随机选择 6 个非零位置
        beta[non_zero_indices] = np.random.choice([-1, 1], 6)  # 非零系数为 1 或 -1

        # 生成特征数据 x
        x = np.random.normal(0, 1, (N, self.num_features))  # 500 维特征，服从标准正态分布

        # 生成噪声 v 和 u
        v = np.random.normal(0, self.sigma_v, N)  # 随机噪声 v
        u = stats.halfnorm.rvs(loc=0, scale=self.sigma_u, size=N)  # 非负噪声 u

        # 生成目标变量 y
        y = x.dot(beta) + v - u

        # 保存真实参数到 CSV 文件
        true_params = {
            'beta': beta,
            'lambda': self.sigma_u / self.sigma_v,  # lambda = sigma_u / sigma_v
            'sigma_sq': self.sigma_u**2 + self.sigma_v**2  # sigma^2 = sigma_u^2 + sigma_v^2
        }
        pd.DataFrame(true_params).to_csv(save_path, index=False)

        return x, y, beta

    def stochastic_frontier_mle_lasso_bic(self, x, y, r_range=np.logspace(-2, 4, 20), threshold=1e-3, max_iter=10):
        """
        使用自适应Lasso惩罚的随机前沿模型进行变量选择和参数估计，并进行后处理筛选和重新估计。
        :param x: 特征矩阵
        :param y: 目标变量
        :param r_range: Lasso惩罚参数的范围
        :param threshold: 系数阈值，小于该值的系数设为0
        :param max_iter: 最大迭代次数
        :return: 估计的beta, lambda, sigma^2, MSE, BIC, 最优r
        """
        x_df = pd.DataFrame(x)
        y_series = pd.Series(y)

        def calculate_bic(r, x_df, y_series):
            # 第一次Lasso估计
            lasso = Lasso(alpha=r)
            lasso.fit(x_df, y_series)
            beta_init = lasso.coef_

            # 计算自适应Lasso权重
            w = np.zeros_like(beta_init)
            for j in range(len(beta_init)):
                if np.abs(beta_init[j]) > threshold:  # 使用阈值判断是否为非零系数
                    w[j] = 1 / np.abs(beta_init[j])
                else:
                    w[j] = 0  # 对接近0的系数赋予0权重

            def logLikFun(param):
                parlab = param[:-2]
                parsigmaSq = param[-2]
                parlambda = param[-1]
                epsilon = y_series - np.dot(x_df, parlab)
                # 添加自适应Lasso惩罚项
                penalty = r * np.sum(w * np.abs(parlab))
                return -np.sum(0.5 * np.log(parsigmaSq) + 0.5 / parsigmaSq * epsilon**2 -
                               norm.logcdf(-epsilon * parlambda / np.sqrt(parsigmaSq))) + penalty

            # 使用最小二乘法获取初始参数
            ols = sm.OLS(y_series, x_df).fit()
            init_params = np.append(ols.params.values, [0.5, sum(ols.resid**2) / (len(y_series) - len(ols.params))])

            # 极大似然估计
            result = minimize(lambda params: -logLikFun(params), init_params, method='Nelder-Mead')

            # 提取估计的参数
            beta = result.x[:-2]  # 提取 beta 参数
            sigma_sq = result.x[-2]  # 提取 sigma^2
            lambda_ = result.x[-1]  # 提取 lambda

            # 后处理：将绝对值小于阈值的系数设为0
            beta[np.abs(beta) < threshold] = 0

            # 计算对数似然值
            log_lik = -logLikFun(result.x)

            # 计算BIC
            n = len(y_series)  # 样本数量
            k = np.sum(beta != 0) + 2  # 非零参数数量（beta + sigma_sq + lambda）
            bic = -2 * log_lik + k * np.log(n)

            return bic, beta, lambda_, sigma_sq

        # 初始化
        best_bic = np.inf
        best_r = None
        best_beta = None
        best_lambda = None
        best_sigma_sq = None
        best_features = None

        # 遍历 r_range，选择使BIC最小的 r
        for r in r_range:
            # 初始估计
            bic, beta, lambda_, sigma_sq = calculate_bic(r, x_df, y_series)

            # 迭代筛选和重新估计
            for _ in range(max_iter):
                # 筛选出非零特征
                selected_features = np.abs(beta) >= threshold
                if not np.any(selected_features):
                    break  # 如果所有特征都被筛除，停止迭代

                # 确保 selected_features 的长度与 x_df 的列数一致
                if len(selected_features) != x_df.shape[1]:
                    selected_features = np.append(selected_features, [False] * (x_df.shape[1] - len(selected_features)))

                # 使用筛选后的特征重新估计
                x_filtered = x_df.loc[:, selected_features]
                bic_new, beta_new, lambda_new, sigma_sq_new = calculate_bic(r, x_filtered, y_series)

                # 如果BIC没有改善，停止迭代
                if bic_new >= bic:
                    break

                # 更新结果
                bic, beta, lambda_, sigma_sq = bic_new, beta_new, lambda_new, sigma_sq_new

            # 记录最优结果
            if bic < best_bic:
                best_bic = bic
                best_r = r
                best_beta = beta
                best_lambda = lambda_
                best_sigma_sq = sigma_sq
                best_features = selected_features

        # 计算 MSE
        epsilon = y_series - np.dot(x_df.loc[:, best_features], best_beta[best_features])
        mse = np.mean(epsilon**2)

        return best_beta, best_lambda, best_sigma_sq, mse, best_bic, best_r

    def train_model_without_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50, threshold=1e-3):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        losses = []
        gradients = []
        parameters = []

        for i in tqdm(range(1, num_iters + 1), desc="Training without privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha(constraint, gradient, model)
            neg_alphas = self.compute_alpha(-constraint, gradient, model)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(sum(p.numel() for p in model.parameters()))
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))  # 记录梯度的范数
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        # 后处理：将绝对值小于阈值的系数设为0
        with torch.no_grad():
            model.beta[torch.abs(model.beta) < threshold] = 0

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def train_model_private(self, x, y, num_iters=3000, constraint=100, minibatch_size=50, lipschitz=1, epsilon=0.1, delta=1e-5, threshold=1e-3):
        x, mean, std = self.standardize_data(x)
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        model = StochasticFrontierModel(num_features=self.num_features)
        n = x.shape[0]
        m = sum(p.numel() for p in model.parameters())
        losses = []
        gradients = []
        parameters = []
        noise_para = lipschitz * constraint * math.sqrt(8 * num_iters * math.log(1 / delta)) / (n * epsilon)

        for i in tqdm(range(1, num_iters + 1), desc="Training with privacy"):
            minibatch_indices = np.random.choice(x.shape[0], minibatch_size, replace=True)
            minibatch_x = x[minibatch_indices]
            minibatch_y = y[minibatch_indices]
            loss = model(minibatch_x, minibatch_y)
            loss.backward()
            gradient = torch.cat([p.grad.flatten() for p in model.parameters()]).detach().numpy()
            pos_alphas = self.compute_alpha_private(constraint, gradient, model, noise_para)
            neg_alphas = self.compute_alpha_private(-constraint, gradient, model, noise_para)
            alphas = pos_alphas + neg_alphas
            min_alpha, size, corner_num = min(alphas, key=lambda x: x[0])
            corner = np.zeros(m)
            corner[corner_num] = size
            mu = 2 / (i + 2)
            with torch.no_grad():
                index = 0
                for p in model.parameters():
                    numel = p.numel()
                    p_flat = p.view(-1)
                    p_flat.copy_((1 - mu) * p_flat + mu * torch.tensor(corner[index:index+numel], dtype=torch.float32))
                    index += numel
            losses.append(loss.item())
            gradients.append(np.linalg.norm(gradient))  # 记录梯度的范数
            parameters.append(model.state_dict().copy())
            model.zero_grad()

        min_loss_index = np.argmin(losses)
        model.load_state_dict(parameters[min_loss_index])

        # 后处理：将绝对值小于阈值的系数设为0
        with torch.no_grad():
            model.beta[torch.abs(model.beta) < threshold] = 0

        return model, mean, std, losses[-1], self.calculate_mse(model, x, y)

    def compute_alpha(self, corner_size, gradient, model):
        alpha = gradient * corner_size
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def compute_alpha_private(self, corner_size, gradient, model, noise_para):
        alpha = gradient * corner_size
        noise = np.random.laplace(scale=noise_para, size=sum(p.numel() for p in model.parameters()))
        alpha = alpha + noise
        corner_size = (np.ones(sum(p.numel() for p in model.parameters())) * corner_size).tolist()
        corner_num = np.arange(sum(p.numel() for p in model.parameters())).tolist()
        return list(zip(alpha, corner_size, corner_num))

    def calculate_mse(self, model, x, y):
        with torch.no_grad():
            predictions = model.predict(x)
            mse = torch.mean((predictions - y) ** 2).item()
        return mse

    def calculate_sse(self, beta_hat, beta_true):
        return np.sum((beta_hat - beta_true) ** 2)

    def calculate_fp(self, beta_hat, beta_true):
        return np.sum((beta_true == 0) & (beta_hat != 0))

    def calculate_sr(self, beta_hat, beta_true, threshold=1e-3):
        return np.linalg.norm(beta_hat - beta_true) <= threshold

    def optimize_hyperparameters(self, x, y, method, num_trials=3):
        def objective(trial):
            # 公共超参数
            constraint = trial.suggest_float('constraint', 1, 100)
            minibatch_size = trial.suggest_int('minibatch_size', 10, 100)
            num_iters = trial.suggest_int('num_iters', 1000, 5000)

            if method == 'private':
                # 私有方法特有的超参数
                lipschitz = trial.suggest_float('lipschitz', 0.1, 1)
                model, _, _, loss, mse = self.train_model_private(
                    x, y, num_iters=num_iters, constraint=constraint, 
                    minibatch_size=minibatch_size, lipschitz=lipschitz
                )
            else:
                # 非私有方法不需要 lipschitz 参数
                model, _, _, loss, mse = self.train_model_without_private(
                    x, y, num_iters=num_iters, constraint=constraint, 
                    minibatch_size=minibatch_size
                )
            return loss

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=num_trials)
        return study.best_params

    def run_experiments(self, N_values, epsilon_values, num_repeats=100):
        results = []
        for N in N_values:
            # 生成数据，并保存真实参数
            true_params_path = f'true_parameters_N{N}.csv'
            x, y, true_beta = self.generate_data(N, save_path=true_params_path)

            # 划分训练集和测试集
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
            pd.DataFrame(np.hstack([x, y.reshape(-1, 1)]), columns=[f'x{i}' for i in range(self.num_features)] + ['y']).to_csv(f'data_{N}.csv', index=False)

            for epsilon in epsilon_values:
                for method in ['mle', 'without_private', 'private']:
                    if method == 'mle':
                        # MLE 方法使用自适应 Lasso 进行变量选择
                        beta, lambda_, sigma_sq, mse, bic, best_r = self.stochastic_frontier_mle_lasso_bic(x_train, y_train)
                        # 计算 SSE, FP, SR
                        sse = self.calculate_sse(beta, true_beta)
                        fp = self.calculate_fp(beta, true_beta)
                        sr = self.calculate_sr(beta, true_beta)
                        hyperparams = {'best_r': best_r}
                    else:
                        # 使用交叉验证优化超参数
                        hyperparams = self.optimize_hyperparameters(x_train, y_train, method)
                        beta_list, lambda_list, sigma_sq_list, mse_list, loss_list = [], [], [], [], []
                        sse_list, fp_list, sr_list = [], [], []
                        for _ in range(num_repeats):
                            if method == 'private':
                                model, _, _, loss, mse = self.train_model_private(x_train, y_train, **hyperparams, epsilon=epsilon)
                            else:
                                model, _, _, loss, mse = self.train_model_without_private(x_train, y_train, **hyperparams)
                            beta_hat = model.beta.detach().numpy()
                            beta_list.append(beta_hat)
                            lambda_list.append(torch.exp(model.log_lambda0).item())
                            sigma_sq_list.append(torch.exp(model.log_sigma2).item())
                            # 在测试集上计算 MSE
                            with torch.no_grad():
                                y_pred = model.predict(torch.tensor(x_test, dtype=torch.float32)).numpy()
                                test_mse = np.mean((y_test - y_pred) ** 2)
                            mse_list.append(test_mse)
                            loss_list.append(loss)
                            # 计算 SSE, FP, SR
                            sse_list.append(self.calculate_sse(beta_hat, true_beta))
                            fp_list.append(self.calculate_fp(beta_hat, true_beta))
                            sr_list.append(self.calculate_sr(beta_hat, true_beta))
                        # 计算平均值
                        beta = np.mean(beta_list, axis=0)
                        lambda_ = np.mean(lambda_list)
                        sigma_sq = np.mean(sigma_sq_list)
                        mse = np.mean(mse_list)
                        loss = np.mean(loss_list)
                        sse = np.mean(sse_list)
                        fp = np.mean(fp_list)
                        sr = np.mean(sr_list)
    
                    # 保存结果
                    results.append({
                        'N': N,
                        'epsilon': epsilon,
                        'method': method,
                        'beta': beta,
                        'lambda': lambda_,
                        'sigma_sq': sigma_sq,
                        'mse': mse,
                        'loss': loss,
                        'sse': sse,
                        'fp': fp,
                        'sr': sr,
                        'hyperparams': hyperparams
                    })
        # 保存结果到 CSV
        pd.DataFrame(results).to_csv('result.csv', index=False)

# 可视化函数
def plot_beta_comparison(result_path, true_params_path, threshold=1e-3):
    """
    可视化真实值与估计值的对比图，高亮被选择的变量。
    :param result_path: 包含估计结果的 CSV 文件路径
    :param true_params_path: 包含真实参数的 CSV 文件路径
    :param threshold: 变量选择的阈值
    """
    # 读取估计结果和真实参数
    result_df = pd.read_csv(result_path)
    true_params_df = pd.read_csv(true_params_path)

    # 提取真实 beta 和估计 beta
    true_beta = true_params_df['beta'].values
    estimated_beta = result_df['beta'].values

    # 确保 beta 的维度一致
    if len(true_beta) != len(estimated_beta):
        raise ValueError("真实 beta 和估计 beta 的维度不一致！")

    # 创建 DataFrame 用于绘图
    beta_df = pd.DataFrame({
        'Dimension': np.arange(len(true_beta)),  # 横轴：维度
        'True Beta': true_beta,  # 纵轴：真实值
        'Estimated Beta': estimated_beta,  # 纵轴：估计值
        'Selected': (np.abs(true_beta) > threshold)  # 高亮被选择的变量
    })

    # 设置 Seaborn 风格
    sns.set(style="whitegrid", font_scale=1.2)
    plt.figure(figsize=(12, 6))

    # 绘制真实值和估计值的散点图
    sns.scatterplot(x='Dimension', y='True Beta', data=beta_df, color='blue', label='True Beta', s=100, alpha=0.7)
    sns.scatterplot(x='Dimension', y='Estimated Beta', data=beta_df, color='red', label='Estimated Beta', s=100, alpha=0.7)

    # 绘制差异线
    for i in range(len(beta_df)):
        plt.plot([beta_df['Dimension'][i], beta_df['Dimension'][i]],
                 [beta_df['True Beta'][i], beta_df['Estimated Beta'][i]],
                 color='gray', linestyle='--', alpha=0.5)

    # 高亮被选择的变量
    selected_points = beta_df[beta_df['Selected']]
    sns.scatterplot(x='Dimension', y='True Beta', data=selected_points, color='green', label='Selected Variables', s=150, edgecolor='black', linewidth=1.5)

    # 添加标题和标签
    plt.title('Comparison of True Beta and Estimated Beta with Variable Selection', fontsize=16)
    plt.xlabel('Dimension', fontsize=14)
    plt.ylabel('Beta Value', fontsize=14)
    plt.legend(loc='upper right', fontsize=12)

    # 调整布局
    plt.tight_layout()
    plt.show()

# 示例调用
num_features = 500  # 特征维度为 500
sigma_u = 0.3  # 非负噪声的标准差
sigma_v = 0.5  # 随机噪声的标准差
sfa = StochasticFrontierAnalysis(num_features, sigma_u, sigma_v)
sfa.run_experiments(N_values=[5000], epsilon_values=[1])

# 可视化结果
result_path = 'result.csv'  # 包含估计结果的 CSV 文件路径
true_params_path = 'true_parameters_N5000.csv'  # 包含真实参数的 CSV 文件路径
plot_beta_comparison(result_path, true_params_path)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def plot_beta_comparison(csv_path, true_beta_path, threshold=1e-3):
    """
    可视化真实值与估计值的对比图，高亮被选择的变量。
    :param csv_path: 包含估计 beta 的 CSV 文件路径
    :param true_beta_path: 包含真实 beta 的 CSV 文件路径
    :param threshold: 变量选择的阈值
    """
    # 读取估计的 beta 和真实的 beta
    df = pd.read_csv(csv_path)
    true_beta = pd.read_csv(true_beta_path).values.flatten()  # 真实 beta
    estimated_beta = df['beta'].values  # 估计的 beta

    # 确保 beta 的维度一致
    if len(true_beta) != len(estimated_beta):
        raise ValueError("真实 beta 和估计 beta 的维度不一致！")

    # 创建 DataFrame 用于绘图
    beta_df = pd.DataFrame({
        'Dimension': np.arange(len(true_beta)),  # 横轴：维度
        'True Beta': true_beta,  # 纵轴：真实值
        'Estimated Beta': estimated_beta,  # 纵轴：估计值
        'Selected': (np.abs(true_beta) > threshold)  # 高亮被选择的变量
    })

    # 设置 Seaborn 风格
    sns.set(style="whitegrid", font_scale=1.2)
    plt.figure(figsize=(12, 6))

    # 绘制真实值和估计值的散点图
    sns.scatterplot(x='Dimension', y='True Beta', data=beta_df, color='blue', label='True Beta', s=100, alpha=0.7)
    sns.scatterplot(x='Dimension', y='Estimated Beta', data=beta_df, color='red', label='Estimated Beta', s=100, alpha=0.7)

    # 绘制差异线
    for i in range(len(beta_df)):
        plt.plot([beta_df['Dimension'][i], beta_df['Dimension'][i]],
                 [beta_df['True Beta'][i], beta_df['Estimated Beta'][i]],
                 color='gray', linestyle='--', alpha=0.5)

    # 高亮被选择的变量
    selected_points = beta_df[beta_df['Selected']]
    sns.scatterplot(x='Dimension', y='True Beta', data=selected_points, color='green', label='Selected Variables', s=150, edgecolor='black', linewidth=1.5)

    # 添加标题和标签
    plt.title('Comparison of True Beta and Estimated Beta with Variable Selection', fontsize=16)
    plt.xlabel('Dimension', fontsize=14)
    plt.ylabel('Beta Value', fontsize=14)
    plt.legend(loc='upper right', fontsize=12)

    # 调整布局
    plt.tight_layout()
    plt.show()

# 示例调用
csv_path = 'result.csv'  # 包含估计 beta 的 CSV 文件路径
true_beta_path = 'true_beta.csv'  # 包含真实 beta 的 CSV 文件路径
plot_beta_comparison(csv_path, true_beta_path)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# import scienceplots

# # 设置 scienceplots 样式
# plt.style.use(['science', 'ieee'])

# # 创建数据
# np.random.seed(0)
# dimensions = np.arange(1, 501)
# true_coefficient = np.zeros(500)
# estimated_values = np.random.normal(0, 0.2, 500)

# # 添加几个异常值以匹配图中的情况
# true_coefficient[[200, 300, 400, 499]] = [1, -1, 1, -1]
# estimated_values[[100, 200, 300, 400, 499]] = [1, 1, -1, 1, -1]

# # 计算差异
# difference = estimated_values - true_coefficient

# # 绘制图形
# plt.figure(figsize=(12, 6))

# # 绘制真实系数
# plt.scatter(dimensions, true_coefficient, marker='D', color='blue', label='True coefficient (beta)', alpha=0.8)

# # 绘制估计值
# plt.scatter(dimensions, estimated_values, marker='s', color='red', label='Estimated values (A-Trans-QMR)', alpha=0.8)

# # 高亮被选择的变量（真实值不为零的点）
# selected_indices = np.where(true_coefficient != 0)[0]
# plt.scatter(dimensions[selected_indices], true_coefficient[selected_indices], 
#             marker='D', color='green', s=100, label='Selected variables', edgecolor='black', linewidth=1.5)

# # 添加差异线
# for i in range(len(dimensions)):
#     plt.plot([dimensions[i], dimensions[i]], [true_coefficient[i], estimated_values[i]], 
#              color='gray', linestyle='--', alpha=0.5)

# # 添加标题和标签
# plt.title('Comparison of True Coefficients and Estimated Values (K=5)', fontsize=16)
# plt.xlabel('Dimension', fontsize=14)
# plt.ylabel('Value', fontsize=14)

# # 添加图例
# plt.legend(fontsize=12, loc='upper right')

# # 显示图形
# plt.tight_layout()
# plt.show()